In [1]:
import os
import yaml

In [2]:
import torch
from torch.utils.data import Subset
import pandas as pd
import torch.nn.functional as F
import pytorch_lightning as pl
from sklearn.model_selection import KFold
from IPython.display import clear_output

In [3]:
from conflab.data_loaders.pose import ConflabPoseExtractor, ConflabToKinetics
from conflab.data_loaders.accel import ConflabAccelExtractor
from conflab.data_loaders.person import ConflabDataset, ConflabSubset, ConflabLabelExtractor
from conflab.baselines.speaking_status.pose.feeders.feeder import Feeder
from conflab.constants import conflab_pose_path, midge_data_path, conflab_speaking_status_path
from utils import count_params, import_class, get_metrics
from trainer import Trainer
from utils import count_params, import_class, init_seed

# Train-test split

In [4]:
def load_data(args):
    def worker_seed_fn(worker_id):
        # give workers different seeds
        return init_seed(args['seed'] + worker_id + 1)

    train_loader = torch.utils.data.DataLoader(
        dataset=Feeder(**args['train_feeder_args']),
        batch_size=args['batch_size'],
        shuffle=True,
        num_workers=args['num_worker'],
        drop_last=True,
        worker_init_fn=worker_seed_fn)

    test_loader = torch.utils.data.DataLoader(
        dataset=Feeder(**args['test_feeder_args']),
        batch_size=args['test_batch_size'],
        shuffle=False,
        num_workers=args['num_worker'],
        drop_last=False,
        worker_init_fn=worker_seed_fn)

    return train_loader, test_loader

In [5]:
defaults = yaml.safe_load(open('./config/defaults.yaml', 'r'))
config = yaml.safe_load(open('./config/conflab/train_joint.yaml', 'r'))
config = {**defaults, **config}
config['phase'] = 'train'
config['work_dir'] = 'transfer_learning/conflab/joint'
config['weights'] = 'pretrained-models/kinetics-joint.pt'
config['ignore_weights'] = ['fc.weight', 'fc.bias']

In [6]:
train_loader, test_loader = load_data(config)

In [7]:
next(iter(train_loader))[0].shape

torch.Size([128, 3, 180, 18, 2])

In [8]:
train_loader, test_loader = load_data(config)
trainer = Trainer(config)
trainer.train(train_loader, test_loader)

log_dir transfer_learning/conflab/joint/trainlogs already exists
Dir removed: transfer_learning/conflab/joint/trainlogs
Cannot parse global_step from model weights filename
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


(BS 64) loss: 1.7460:  57%|█████▋    | 77/136 [00:57<00:43,  1.34it/s]


KeyboardInterrupt: 

# Cross-validation

In [4]:
defaults = yaml.safe_load(open('./config/defaults.yaml', 'r'))
config = yaml.safe_load(open('./config/conflab/train_joint.yaml', 'r'))
config = {**defaults, **config}
config['phase'] = 'train'
config['work_dir'] = 'conflab'
config['weights'] = 'pretrained-models/kinetics-joint.pt'
config['ignore_weights'] = ['fc.weight', 'fc.bias']

In [5]:
# format conflab dataset dict output to msg3d tuple (data, label, index) 
def collate(batch):
    data = [e['pose'] for e in batch]
    labels = [e['label'] for e in batch]
    idxs = [e['index'] for e in batch]

    return torch.stack(data), torch.tensor(labels), torch.tensor(idxs)

In [6]:
def do_fold(train_ds, test_ds, deterministic=False):
    # data loaders
    data_loader_train = torch.utils.data.DataLoader(
        train_ds, batch_size=128, shuffle=True, num_workers=4,
        collate_fn=collate, drop_last=True)
    data_loader_val = torch.utils.data.DataLoader(
        test_ds, batch_size=128, shuffle=False, num_workers=4,
        collate_fn=collate, drop_last=True)

    trainer = Trainer(config)
    trainer.train(data_loader_train, data_loader_val)

    return trainer.test(data_loader_val)

In [7]:
def do_run(dataset, random_state, metrics_name='binary', deterministic=False):
    cv_splits = KFold(n_splits=4, random_state=random_state, shuffle=True).split(range(len(ds)))

    metrics = []
    scores = []
    for f, (train_idx, test_idx) in enumerate(cv_splits):
        # create datasets    
        train_ds = ConflabSubset(dataset, train_idx)
        test_ds = ConflabSubset(dataset, test_idx)

        fold_metrics, fold_scores = do_fold(train_ds, test_ds, deterministic=deterministic)
        clear_output(wait=True)
        metrics.append(fold_metrics)
        scores.append(fold_scores)

    return metrics, scores

In [8]:
pose_extractor = ConflabPoseExtractor(conflab_pose_path)
pose_extractor.load_from_pickle('../tracks.pkl')
label_extractor = ConflabLabelExtractor(os.path.join(conflab_speaking_status_path, 'speaking'))

In [9]:
# make windowed examples using the pose tracks.
examples = pose_extractor.make_examples()
# compose the dataset
ds = ConflabDataset(examples, {
    'pose': pose_extractor,
    'label': label_extractor
}, transform=ConflabToKinetics())

100%|██████████| 8/8 [00:00<00:00, 544.49it/s]


In [10]:
seed=22
pl.utilities.seed.seed_everything(seed, workers=True)
metrics, scores = do_run(ds, random_state=seed, metrics_name='binary', deterministic=True)

Global seed set to 22


logdir is transfer_learning/conflab/joint/trainlogs
log_dir transfer_learning/conflab/joint/trainlogs already exists
Dir removed: transfer_learning/conflab/joint/trainlogs
Cannot parse global_step from model weights filename
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


(BS 64) loss: 0.6167:  22%|██▏       | 54/251 [00:36<02:11,  1.49it/s]


KeyboardInterrupt: 

In [11]:
metrics

[{'accuracy': 0.6808035714285714,
  'loss': 1.958919610295977,
  'auc': 0.6655879003830165},
 {'accuracy': 0.6138392857142857,
  'loss': 2.812851599284581,
  'auc': 0.6278296578440496},
 {'accuracy': 0.6450892857142857,
  'loss': 1.9170774562018258,
  'auc': 0.6392705881043416},
 {'accuracy': 0.6183035714285714,
  'loss': 3.0535796540124074,
  'auc': 0.5869259331639632}]